In [10]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [11]:
llm = init_chat_model("google_genai:gemini-2.0-flash")
    
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [15]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
response = graph.invoke({"messages": [message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='b459b640-28fb-40de-ad75-d6096c041f3d'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6bd5f18e-3114-4b7b-8ca9-9527a7c477d4-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [18]:
state = None
while True:
    in_messages = input("You: ")
    if in_messages.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_messages}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_messages})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: Neil Armstrong was the first person to walk on the Moon. He took his first steps on the lunar surface on July 20, 1969, as part of the Apollo 11 mission.
